# Disponibilité mondiale des produits végétaux en kcal/kg

In [8]:
import pandas as pd

In [9]:
#Récupération des données utiles
data_population = pd.read_csv('fr_population.csv')
data_vegetaux = pd.read_csv('fr_vegetaux.csv')

#On gère nos noms de colonnes via des variables pour faciliter la maintenance du code
code_zone = "Code zone"
zone = 'Zone'
valeur = 'Valeur'
element = "Élément"
code_domaine = 'Code Domaine'
domaine = 'Domaine'
code_element = 'Code Élément'
code_produit = 'Code Produit'
produit = 'Produit'
code_annee = 'Code année'
annee = 'Année'
symbole = 'Symbole'
description_symbole = 'Description du Symbole'
unite = 'Unité'
energie_poids = 'Ratio énergie/poids en Kcal/Kg'
valeur_kg = 'Valeur en kg'
energie_poids_dispo_int = 'Ratio énergie/poids pour la disponibilité intérieure en kcal/kg '

data_population = data_population[data_population[code_zone] != 351]
data_vegetaux = data_vegetaux[data_vegetaux[code_zone] != 351]
data_vegetaux = data_vegetaux.drop([code_zone, code_element, code_domaine, domaine, code_produit, code_annee, annee, symbole, description_symbole], axis = 1)
data_vegetaux.shape

(104011, 5)

In [10]:
#Fonction qui récupère la population d'un pays donné
def recup_pop(pays) :
    pop = data_population[data_population[zone] == pays][valeur].values
    pop = int(pop * 1000)
    return pop

In [11]:
#Fonction qui permet de calculer le ratio energie/poids d'un produit dans un pays
def calculer_energie_poids(data) :
    data_ratio_values = []
    mask1 = data[element] == 'Nourriture'
    mask3 = data[element] == 'Disponibilité alimentaire (Kcal/personne/jour)'
    data = data[mask1 | mask3]
    kg = 0
    kcal = 0
    for z, p, u, v in zip(data[zone], data[produit], data[unite], data[valeur]) :
        div = False
        if u == 'Milliers de tonnes' :
            kg = v * 1000000
        else :
            kcal = v
            div = True
        if div == True :
            if kg == 0 :
                ratio = 0
            else :
                pop = recup_pop(z)
                kcal *= pop * 365
                ratio = round((kcal / kg), 2)
                ratio_values =  [z, p, ratio]
                data_ratio_values.append(ratio_values)
    data_ratio = pd.DataFrame(data_ratio_values, columns = [zone, produit, energie_poids])
    return data_ratio

In [12]:
#Fonction qui récupère la disponibilité intérieure mondiale de chaque produit en kg
def recuperer_dispo_int(data) :
    mask1 = data[element] == 'Disponibilité intérieure'
    mask2 = data[valeur] >0
    data.loc[:,valeur] *= 1000000
    data = data[mask1 & mask2]
    data = data.rename(columns = {valeur : valeur_kg})
    data = data.groupby(produit).sum(valeur_kg).reset_index()
    return data

In [13]:
#Fonction qui calcul le ratio kcal/disponibilité intérieure mondiale
def calculer_ratio_kcal_dispo_int(data1, data2) :
    #On compare nos dataframe et on supprime les produits dont des données manquent 
    data_1_len = len(data1)
    data_2_len = len(data2)
    col1 = data1.columns.values
    col2 = data2.columns.values
    kcal_dispo_int = []
    #On retire les lignes non renseignées dans les 2 dataframe
    if data_1_len > data_2_len :
        for p in data1[produit] :
            data3 = data2[data2[produit] == p]
            z = len(data3)
            if z == 0 :
                data1 = data1[data1[produit] != p]
            else :
                prod = data1[data1[produit] == p]
                kcal_kg = prod[col1[1]].values * float (data3[col2[1]].values)
                ratio_values = [p, round(kcal_kg, 2)]
                kcal_dispo_int.append(ratio_values)
    else :
        for p in data2[produit] :
            data3 = data1[data1[produit] == p]
            z = len(data3)
            if z == 0 :
                data2 = data2[data2[produit] != p]
            else :
                prod = data2[data2[produit] == p]
                kcal_kg = float (prod[col2[1]].values) * float (data3[col1[1]].values)
                ratio_values = [p, round(kcal_kg, 2)]
                kcal_dispo_int.append(ratio_values)
    data1 = pd.DataFrame(kcal_dispo_int, columns = [produit, energie_poids_dispo_int]).set_index(produit)
    donnee = data1.sum()
    donnee = data1.sum()
    donnee = donnee[0]
    return donnee

In [14]:
data_ratio_kcal = calculer_energie_poids(data_vegetaux)
data_ratio_kcal.head()

,Zone,Produit,Ratio énergie/poids en Kcal/Kg
0,Afghanistan,Blé,3118.77
1,Afghanistan,Riz (Eq Blanchi),3725.97
2,Afghanistan,Orge,3257.74
3,Afghanistan,Maïs,3081.33
4,Afghanistan,Millet,2787.87


In [15]:
#On fait la moyenne du ratio energie/kg pour chaque produit et on les classe de façon décroissante
data_ratio_kcal = data_ratio_kcal.groupby(produit).mean(energie_poids).reset_index()
data_ratio_kcal.head()

,Produit,Ratio énergie/poids en Kcal/Kg
0,"Agrumes, Autres",90.316765
1,Aliments pour enfants,3224.621684
2,Ananas,366.682759
3,Arachides Decortiquees,5620.249638
4,Avoine,1892.484894


In [16]:
data_dispo_int = recuperer_dispo_int(data_vegetaux)
data_dispo_int.head()

,Produit,Valeur en kg
0,"Agrumes, Autres",1.226300e+10
1,"Alcool, non Comestible",2.176900e+10
2,Aliments pour enfants,7.670000e+08
3,Ananas,2.328300e+10
4,Arachides Decortiquees,2.969500e+10


In [17]:
data_ratio_kcal_dispo_int = calculer_ratio_kcal_dispo_int(data_ratio_kcal, data_dispo_int)

In [20]:
#Disponibilité intérieure mondiale en kcal
print('Disponibilité intérieure mondiale en kcal : ', data_ratio_kcal_dispo_int)

Disponibilité intérieure mondiale en kcal :  1.3032992885918248e+16


In [19]:
#data_ratio_kcal.to_csv('Q6_data_ratio_kcal.csv')
#data_dispo_int.to_csv('Q6_data_dispo_int.csv')